In [48]:
include("altiplano_option.jl")

altiplano_option_monte_carlo (generic function with 2 methods)

In [49]:
cov_matrix = [1.0 0.3 0.4; 0.3 1.0 0.1; 0.4 0.1 1.0] 

normal = [altiplano_option_monte_carlo(1000,0.05,10,0.5,0.01,7.0,10.0,[1,3,7],3,[10.0,10.0,10.0],[0.07,0.07,0.07],[0.06,0.05,0.09],cov_matrix,"basic") for i in 1:10]
antithetic = [altiplano_option_monte_carlo(1000,0.05,10,0.5,0.01,7.0,10.0,[1,3,7],3,[10.0,10.0,10.0],[0.07,0.07,0.07],[0.06,0.05,0.09],cov_matrix,"antithetic") for i in 1:10]
quasi = [altiplano_option_monte_carlo(1000,0.05,10,0.5,0.01,7.0,10.0,[1,3,7],3,[10.0,10.0,10.0],[0.07,0.07,0.07],[0.06,0.05,0.09],cov_matrix,"quasi_monte_carlo") for i in 1:10]
moment_matching = [altiplano_option_monte_carlo(1000,0.05,10,0.5,0.01,7.0,10.0,[1,3,7],3,[10.0,10.0,10.0],[0.07,0.07,0.07],[0.06,0.05,0.09],cov_matrix,"moment_matching") for i in 1:10]
LHS = [altiplano_option_monte_carlo(1000,0.05,10,0.5,0.01,7.0,10.0,[1,3,7],3,[10.0,10.0,10.0],[0.07,0.07,0.07],[0.06,0.05,0.09],cov_matrix,"LHS") for i in 1:10]

mean_elements = mean([normal[i][1] for i in eachindex(normal)])
bottom_elements = mean([normal[i][2] for i in eachindex(normal)])
upper_elements = mean([normal[i][3] for i in eachindex(normal)])

mean_antithetic = mean([antithetic[i][1] for i in eachindex(antithetic)])
bottom_antithetic = mean([antithetic[i][2] for i in eachindex(antithetic)])
upper_antithetic = mean([antithetic[i][3] for i in eachindex(antithetic)])

mean_quasi = mean([quasi[i][1] for i in eachindex(quasi)])
bottom_quasi = mean([quasi[i][2] for i in eachindex(quasi)])
upper_quasi = mean([quasi[i][3] for i in eachindex(quasi)])

mean_moment_matching = mean([moment_matching[i][1] for i in eachindex(moment_matching)])
bottom_moment_matching = mean([moment_matching[i][2] for i in eachindex(moment_matching)])
upper_moment_matching = mean([moment_matching[i][3] for i in eachindex(moment_matching)])

mean_LHS = mean([LHS[i][1] for i in eachindex(LHS)])
bottom_LHS = mean([LHS[i][2] for i in eachindex(LHS)])
upper_LHS = mean([LHS[i][3] for i in eachindex(LHS)])

length_normal = upper_elements-bottom_elements
length_antithetic = upper_antithetic - bottom_antithetic
length_quasi = upper_quasi-bottom_quasi
length_moment_matching = upper_moment_matching - bottom_moment_matching
length_LHS = upper_LHS-bottom_LHS


println("basic mean : $mean_elements length: $length_normal")
println("antithetic mean : $mean_antithetic length antithetic: $length_antithetic")
println("quasi mean : $mean_quasi length quasi: $length_quasi")
println("moment matching mean : $mean_moment_matching length matching: $length_moment_matching")
println("LHS mean : $mean_LHS length LHS: $length_LHS")

basic mean : 3.6649217477086777 length: 0.08021060215156117
antithetic mean : 3.662025494612583 length antithetic: 0.007912525128333847
quasi mean : 5.023070887469941 length quasi: 0.0
moment matching mean : 4.183237496364766 length matching: 0.07461726917979838
LHS mean : 4.11646727589153 length LHS: 0.002139336920490287
